In [2]:
import sys
from pathlib import Path
import json

import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib

DIR = Path('..')
sys.path.append(str(DIR))

import gtfs_kit as gk

DATA_DIR = DIR/'data/'

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
path = DATA_DIR / 'C:/Users/jcele/Desktop/capstone_project/Capstone_project/google_transit.zip'

gk.list_feed(path)

,file_name,file_size
0,agency.txt,601
1,calendar.txt,840
2,calendar_dates.txt,2332
3,fare_attributes.txt,596
4,fare_rules.txt,1062
5,routes.txt,19275
6,shapes.txt,9135561
7,stop_times.txt,113923612
8,stops.txt,625351
9,trips.txt,4654932


In [5]:
# Read feed and describe

path = DATA_DIR / 'C:/Users/jcele/Desktop/capstone_project/Capstone_project/google_transit.zip'

feed = gk.read_feed(path, dist_units='km')
feed.describe()

,indicator,value
0,agencies,"[Sound Transit, Metro Transit, City of Seattle]"
1,timezone,America/Los_Angeles
2,start_date,20230417
3,end_date,20230901
4,num_routes,167
5,num_trips,54727
6,num_stops,6663
7,num_shapes,552
8,sample_date,20230420
9,num_routes_active_on_sample_date,166


In [6]:
# Validate

feed.validate()

,type,message,table,rows
0,warning,Unrecognized column agency_id,fare_attributes,[]
1,warning,Unrecognized column fare_period_id,fare_attributes,[]
2,warning,Unrecognized column descriptions,fare_attributes,[]
3,warning,Unrecognized column peak_flag,trips,[]
4,warning,Unrecognized column fare_id,trips,[]


In [7]:
# Append shape_dist_traveled column to stop times

display(feed.stop_times.head().T)

feed = feed.append_dist_to_stop_times()
feed.stop_times.head().T

,0,1,2,3,4
trip_id,347461003,347461004,347461003,347461004,347461003
arrival_time,05:39:00,05:39:00,05:39:49,05:39:49,05:40:18
departure_time,05:39:00,05:39:00,05:39:49,05:39:49,05:40:18
stop_id,67655,67655,67612,67612,70608
stop_sequence,1,1,11,11,16
stop_headsign,NaN,NaN,NaN,NaN,NaN
pickup_type,0,0,0,0,0
drop_off_type,0,0,0,0,0
shape_dist_traveled,0.0,0.0,2045.4,2045.4,3266.3
timepoint,1,1,0,0,0


C:\Users\jcele\AppData\Roaming\Python\Python311\site-packages\shapely\linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
C:\Users\jcele\AppData\Roaming\Python\Python311\site-packages\shapely\linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
C:\Users\jcele\AppData\Roaming\Python\Python311\site-packages\shapely\linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
C:\Users\jcele\AppData\Roaming\Python\Python311\site-packages\shapely\linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
C:\Users\jcele\AppData\Roaming\Python\Python311\site-packages\shapely\linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
C:\Users\jcele\AppData\Roaming\Python\Python3

,0,1,2,3,4
trip_id,347461003,347461003,347461003,347461003,347461003
arrival_time,05:39:00,05:39:49,05:40:18,05:40:34,05:40:50
departure_time,05:39:00,05:39:49,05:40:18,05:40:34,05:40:50
stop_id,67655,67612,70608,70619,72932
stop_sequence,1,11,16,20,23
stop_headsign,NaN,NaN,NaN,NaN,NaN
pickup_type,0,0,0,0,0
drop_off_type,0,0,0,0,0
timepoint,1,0,0,0,0
shape_dist_traveled,0.000156,0.622098,0.994226,1.19696,1.396041


In [8]:
# Choose study dates

week = feed.get_first_week()
dates = [week[4], week[6]]  # First Friday and Sunday
dates

['20230421', '20230423']

In [9]:
# Compute feed time series

trip_stats = feed.compute_trip_stats()
trip_stats.head().T
fts = feed.compute_feed_time_series(trip_stats, dates, freq='6H')
fts


indicator,num_trip_ends,num_trip_starts,num_trips,service_distance,service_duration,service_speed
datetime,,,,,,
2023-04-21 00:00:00,518.0,816.0,811.0,10849.496756,401.816667,27.001112
2023-04-21 06:00:00,4203.0,4347.0,4638.0,69869.529683,2983.816667,23.416160
2023-04-21 12:00:00,4191.0,4373.0,4805.0,69480.715749,3259.616667,21.315609
2023-04-21 18:00:00,3070.0,2556.0,3177.0,44126.264043,1858.350000,23.744862
2023-04-22 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-22 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-22 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-22 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-23 00:00:00,303.0,391.0,389.0,5532.123929,213.683333,25.889356
